# HashiCorp Vault Demo for Secrets Sync features.

HashiCorp Vault's Secrets Sync feature manages the entire lifecycle of the secret and the secret is synchronized to the desired destination provider.  Applications will still be able to natively into the desired destination provider of choice.  This provides an easy way for applications to get the Zero Trust benefits of Vault without having to directly integrate with Vault.  i.e. Centralized secret inventory, audit and reporting, and automation of credential rotation.

Supported destinations include
- AWS Secrets Manager
- Azure Key Vault
- GCP Secret Manager
- GitHub Repository Actions
- Vercel Projects
Ref: https://developer.hashicorp.com/vault/docs/sync#destinations

<img src="images/vault-demo-secrets-sync-aws.png">

In this demo, we will show how key/value secrets that are stored in Vault's KV v2 Secret Engine are actively synchronized into an external target destination used by applications. If the secret value is updated in Vault, the secret is updated in the destination as well. If the secret is deleted from Vault, it is deleted on the external system as well. This process is asynchronous and event-based. Vault propagates modifications into the proper destinations automatically in a few seconds.

To run this notebook in VS Code, chose the Jupyter kernel and then Bash

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

This also assumes you have the Vault CLI installed.

You will need Vault to be installed with an Enterprise license.

# Setting Up the Vault

In [ ]:
# For HashiCorp staff only.  Log in via doormat and populate your AWS credentials into your environment variables.
doormat login -f && eval $(doormat aws export --role $(doormat aws list | grep -m 1 role))

# For normal usage, you will setup the AWS IAM programmatic credentials that contain the permissions to secrets manager
# Ref: https://developer.hashicorp.com/vault/docs/sync/awssm#setup

# Make sure the AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and AWS_SESSION_TOKEN environment variables are populated
echo
echo "AWS_ACCESS_KEY_ID: ${AWS_ACCESS_KEY_ID}"
echo "AWS_SECRET_ACCESS_KEY: ${AWS_SECRET_ACCESS_KEY}"
echo "AWS_SESSION_TOKEN: ${AWS_SESSION_TOKEN}"

In [ ]:
# For this demo, we will be passing our doormat credentials as environment variables.
# For non-doormat scenarios, use your configured IAM programmatic credentials in the secret sync configuration later on.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)

# Refresh Vault docker image with latest version
#docker pull hashicorp/vault-enterprise

# Run Vault in docker in Dev mode with Enterprise license.
# We have set VAULT_LOG_LEVEL to trace for troubleshooting purposes.  This will allow you to view detailed information as you test.
# For doormat users, we will be using environment variables for the AWS credentials as we need to use the session token.
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=trace" \
-e "AWS_ACCESS_KEY_ID=${AWS_ACCESS_KEY_ID}" \
-e "AWS_SECRET_ACCESS_KEY=${AWS_SECRET_ACCESS_KEY}" \
-e "AWS_SESSION_TOKEN=${AWS_SESSION_TOKEN}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# If you have IAM programmatic credentials for the secret sync, use this command instead
# and set the AWS credentials in the secret sync configuration.
#docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
#-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
#-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
#-e "VAULT_LICENSE=${VAULT_LICENSE}" \
#-e "VAULT_LOG_LEVEL=trace" \
#-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest


# Set up secrets sync with AWS Secrets Manager

Ref:
- https://developer.hashicorp.com/vault/docs/sync/awssm

In [ ]:
# Enable KV v2 secret engine
#This will be used to show the sync to the external secrets engine.
export KV_V2_PATH=demo-kv
vault secrets enable -path=$KV_V2_PATH kv-v2

In [ ]:
# Simulate storing a database secret in the KV v2 secret engine
# We will be generating random values for the userid and password
export MY_SECRET_NAME=demo-database-secret
vault kv put -mount=$KV_V2_PATH $MY_SECRET_NAME \
  userid="svc-account-$(openssl rand -base64 10 | tr -dc 'a-zA-Z0-9')" \
  password="$(openssl rand -base64 20 | tr -dc 'a-zA-Z0-9')"

In [ ]:
# Write the configuration for the sync settings
vault write sys/sync/destinations/aws-sm/my-awssm-1 \
      region='ap-southeast-1'

# Use this command when you have the AWS IAM programmatic keys
#vault write sys/sync/destinations/aws-sm/my-awssm-1 \
#      access_key_id="$AWS_ACCESS_KEY_ID" \
#      secret_access_key="$AWS_SECRET_ACCESS_KEY" \
#      region='ap-southeast-1'


In [ ]:
# Configure the sync to the secret
echo "KV v2 Path: $KV_V2_PATH"
echo "Secret name: $MY_SECRET_NAME"
vault write sys/sync/destinations/aws-sm/my-awssm-1/associations/set \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"

# Check in your AWS Console that the secret has been created
# verify that the secret values match
vault kv get $KV_V2_PATH/$MY_SECRET_NAME

In [ ]:
# Update the secret value
vault kv put -mount=$KV_V2_PATH $MY_SECRET_NAME \
  userid="svc-account-$(openssl rand -base64 10 | tr -dc 'a-zA-Z0-9')" \
  password="$(openssl rand -base64 20 | tr -dc 'a-zA-Z0-9')"

# Verify that the secret values match
vault kv get $KV_V2_PATH/$MY_SECRET_NAME

# Refresh AWS secrets manager


# Cleanup

In [ ]:
# Cleanup AWS Secrets Manager

# Remove association
vault write sys/sync/destinations/aws-sm/my-awssm-1/associations/remove \
    mount="$KV_V2_PATH" \
    secret_name="$MY_SECRET_NAME"
# Delete secret sync destination
vault delete sys/sync/destinations/aws-sm/my-awssm-1

In [ ]:
# Stop Vault container
docker stop vault-enterprise